In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import time
import matplotlib.pyplot as plt
from numpy import savez_compressed
import math
from collections import Counter

%matplotlib inline

In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# print('Found GPU at: {}'.format(device_name))

In [ ]:
### num_epochs now = 2

class ModelConfig:
    EPOCHS = 10
    BATCH_SIZE = 4
    num_frames_per_second = 10
    SEQUENCE_SIZE = 16
    H = 256
    W = 256
    C = 3
    TO_GRAY = False
    overlapping = 0
    rootdir = "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos"
    TRAIN_SAMPLE_NPZ_DIRECTORY = "./train_npz_files"
    TEST_SAMPLE_NPZ_DIRECTORY = "./test_npz_files"
    types = {"Normal":0, "Abnormal":1}
    classes = {"Explosion":1, 'Burglary':2, 'Fighting':3, 'Assault':4, 'Arrest':5, 'Arson':6, 'Abuse':7}
    extension = "mp4"
    NUM_CLASSES = 8
    MODEL_WEIGHTS_DIRECTORY = "./model_weights"
    COMBINE_MODEL_PATH = "combined_model_weights.hdf5"
    GENERATOR_MODEL_PATH = "generator_model_weights.hdf5"
    DISCRIMINATOR_MODEL_PATH = "discriminator_model_weights.hdf5"
    CLASSIFIER_MODEL_PATH = "classifier_model_weights.hdf5"
    AUTOENCODER_MODEL_PATH = "autoencoder_model.hdf5"
    
    NUM_SEGMENTS = 10
    DIFF_BETWEEN_SEGMENTS = 5


In [ ]:
os.mkdir(ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)
os.mkdir(ModelConfig.TEST_SAMPLE_NPZ_DIRECTORY)
os.mkdir(ModelConfig.MODEL_WEIGHTS_DIRECTORY)


In [ ]:
def get_video_times(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    print('fps = ' + str(fps))
    print('number of frames = ' + str(frame_count))
    print('duration (S) = ' + str(duration))
    minutes = int(duration / 60)
    seconds = duration % 60
    print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))


In [ ]:
def crop_image_black_background(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    hh, ww = thresh.shape
    thresh[hh:hh, 0:ww] = 0
    white = np.where(thresh == 255)
    
    if len(white[0]) <= 1 or len(white[1]) <= 1:
        return None
    
    xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])
    crop = img[ymin:ymax, xmin:xmax]
    
    if (crop.shape[0] < img.shape[0]/2) and (crop.shape[1] < img.shape[1]/2):
        return None
    return crop

In [ ]:
def SaveVideo2Npz(file_path, npz_directory, resize=(ModelConfig.H, ModelConfig.W), 
                  num_target_frames=ModelConfig.SEQUENCE_SIZE, overlapping=0):

    cap = cv2.VideoCapture(file_path)
    file_name = file_path.split('/')[-1]
    file_name = file_name.split('.')[0]
    fps = cap.get(cv2.CAP_PROP_FPS)
    len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    skip_length = int((len_frames-(ModelConfig.NUM_SEGMENTS*ModelConfig.DIFF_BETWEEN_SEGMENTS)) / ModelConfig.SEQUENCE_SIZE)   
           
    segments_path_list = []
    frames = []
    try:        
        for seg_num in range(ModelConfig.NUM_SEGMENTS):
            cap_v = cv2.VideoCapture(file_path)
            len_frames = int(cap_v.get(cv2.CAP_PROP_FRAME_COUNT))            
            next_index = seg_num*ModelConfig.DIFF_BETWEEN_SEGMENTS
            first_segt = True
            index = 0
            while index < len_frames:
                _, frame = cap_v.read()              
                
                if index == next_index:
                    next_index = index+skip_length
                
                    #frame = crop_image_black_background(frame)
                    #if frame is None or (frame.shape[0] <= 0 or frame.shape[1] <= 0):
                    #    continue
                    
                    frame = cv2.resize(frame, resize, interpolation=cv2.INTER_AREA)           
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = np.array(frame, dtype=np.float32)
                    frame /= 255.0
                    frames.append(frame)
                index+=1
                
            cap_v.release()
            segments_path = os.path.join(npz_directory, file_name + "_{}.npz".format(seg_num))
            if len(frames) > ModelConfig.SEQUENCE_SIZE:
                frames = frames[(len(frames)-ModelConfig.SEQUENCE_SIZE):]
                
            #print(np.array(frames).shape)
            savez_compressed(segments_path, np.array(frames))
            frames.clear()
            segments_path_list.append(segments_path)     
    except Exception as e:
        print(e)
         
    return np.array(segments_path_list), fps

In [ ]:
skip_files = [
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary064_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Explosion/Explosion046_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/Arrest047_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Fighting/Fighting041_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arson/Arson019_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_940_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary095_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_935_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_924_x264.mp4",
#     "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/Arrest049_x264.mp4"
]

In [ ]:
# segments, _ = SaveVideo2Npz("../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary001_x264.mp4",
#                             npz_directory=ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY,
#                             overlapping=ModelConfig.overlapping)

# print(segments)

In [ ]:
def read_npz_file(file_path):
    dict_data = np.load(file_path)
    data = dict_data['arr_0']
    return data

In [ ]:
# X = read_npz_file("./npz_files/Burglary001_x264_0.npz")
# plt.imshow(X[20])

In [ ]:
# read_npz_file("./npz_files/Burglary008_x264_0.npz").shape

In [ ]:
def clear_npz_directory(directorty_path):
    for root_f in os.listdir(directorty_path):
        f1 = os.path.join(directorty_path, root_f)
        for sub_f in os.listdir(f1):
            os.remove(os.path.join(f1, sub_f))
        
clear_npz_directory(ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)     


In [ ]:

file_name_list = []
file_path_list = []
file_type_list = []
file_class_list = []
file_npy_path_list = []


In [ ]:
def build_dataset_df():
    global file_name_list 
    global file_path_list
    global file_type_list 
    global file_class_list 
    global file_npy_path_list

    file_name_list.clear()
    file_path_list.clear()
    file_type_list.clear()
    file_class_list.clear()
    file_npy_path_list.clear()


    for root, subdirs, files in os.walk(ModelConfig.rootdir):
        for filename in files:
            if filename.split('.')[-1] != ModelConfig.extension:
                continue

            file_path = os.path.join(root, filename)
            file_name = filename.split('.')[0]
            
            if file_path in skip_files:
                continue
            
            file_name_list.append(file_name)
            file_path_list.append(file_path)
            file_class = file_path.split('/')[-2]
            #print(file_class)
            if file_class in ModelConfig.classes.keys():
                file_type_list.append(ModelConfig.types["Abnormal"])
                file_class_list.append(ModelConfig.classes[file_class])
            else:
                file_type_list.append(ModelConfig.types["Normal"])
                file_class_list.append(ModelConfig.types["Normal"])

            #npy_path = Save2Npy(file_path, file_name, Config.save_npy_dir)
            #file_npy_path_list.append(npy_path)


In [ ]:
build_dataset_df()


In [ ]:
len(file_path_list), len(file_name_list), len(file_type_list), len(file_class_list)#, len(file_npy_path_list)


In [ ]:

#dataset_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list, file_npy_path_list)),
#                          columns =['file_name', 'file_path', 'file_type', 'file_class', 'npy_file_path'])

dataset_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list)),
                          columns =['file_name', 'file_path', 'file_type', 'file_class'])


In [ ]:
dataset_df


In [ ]:
dataset_df["file_class"].value_counts()

In [ ]:

import random 

n = random.randint(0, len(dataset_df))
randomlist = random.sample(range(0, len(dataset_df)), len(dataset_df)) 
val_size = 0.1
val_size = int(len(dataset_df) * val_size)
val_idx = randomlist[:val_size]
train_idx = randomlist[val_size:]


In [ ]:
validation_df = dataset_df.iloc[val_idx].reset_index()
train_df = dataset_df.iloc[train_idx].reset_index()

In [ ]:
train_df.to_csv("train_df.csv")
validation_df.to_csv("validation_df.csv")

In [ ]:
def build_dataset(dataframe, npz_directory):
    global file_path_list
    global file_type_list 
    global file_class_list 
    global file_npy_path_list

    file_name_list.clear()
    file_path_list.clear()
    file_type_list.clear()
    file_class_list.clear()
    file_npy_path_list.clear()

    
    for c in np.unique(dataset_df["file_class"]):    
        os.makedirs(os.path.join(npz_directory, str(c)), exist_ok=True)
        
    path_list = []
    class_list = []
    for index, row in dataframe.iterrows():
        print(str(index), " Done")
        sample_file_path = row["file_path"]
        sample_npz_directory = os.path.join(npz_directory, str(row["file_class"]))
        segments, _ = SaveVideo2Npz(sample_file_path, sample_npz_directory)
        for seg_path in segments:
            file_name_list.append(row["file_name"])
            file_path_list.append(row["file_path"])
            file_type_list.append(row["file_type"])
            file_class_list.append(row["file_class"])
            file_npy_path_list.append(seg_path)
            

In [ ]:
build_dataset(train_df, ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)


train_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list, file_npy_path_list)),
                        columns =['file_name', 'file_path', 'file_type', 'file_class', 'file_npy_path_list'])

train_df

In [ ]:
build_dataset(validation_df, ModelConfig.TEST_SAMPLE_NPZ_DIRECTORY)


validation_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list, file_npy_path_list)),
                        columns =['file_name', 'file_path', 'file_type', 'file_class', 'file_npy_path_list'])

validation_df

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(train_df["file_class"]),
                                                 y=train_df["file_class"])

class_weights = {k: v for k,v in enumerate(class_weights)}
class_weights

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, dataset_df, X_col, y_col, shuffle=True):
        self.batch_size = ModelConfig.BATCH_SIZE
        self.dataset_df = dataset_df
        self.X_col = X_col
        self.y_col = y_col
        self.shuffle = shuffle
        self.classes = np.unique(self.dataset_df[self.y_col])
        self.num_classes =  len(self.classes)
        self.X_path, self.Y_dict = self.search_data() 
        self.print_stats()
        self.on_epoch_end()
        return None
        
    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = to_categorical(self.dataset_df[self.y_col], self.num_classes)
        for index in range(len(self.dataset_df)):
            X_path.append(self.dataset_df.at[index, self.X_col])
            Y_dict[X_path[-1]] = one_hots[index]
        return X_path, Y_dict
    
    def print_stats(self):
        self.n_files = len(self.X_path)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        print("Found {} files belonging to {} classes.".format(self.n_files,self.num_classes))
    
    def __len__(self):
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_path = [self.X_path[k] for k in batch_indexs]
        batch_x, batch_y = self.data_generation(batch_path)               
        return batch_x, batch_y
    
    def get_mini_batch(self, index):
        return self.__getitem__(index)

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
    
    def load_data(self, path):
        sample = read_npz_file(path)
        return sample


In [ ]:
train_gen = CustomDataGen(train_df,
                           X_col="file_npy_path_list",
                           y_col="file_class",
                           shuffle=True)


#train_gen.get_mini_batch(0)

In [ ]:
validation_gen = CustomDataGen(validation_df,
                               X_col="file_npy_path_list",
                               y_col="file_class",
                               shuffle=False)


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv3D, BatchNormalization, ReLU, Add, MaxPool3D, GlobalAveragePooling3D, Concatenate, Dropout, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input


def Conv_BN_ReLU(planes, kernel_size, strides=(1, 1, 1), padding='same', use_bias=False):
    return Sequential([
        Conv3D(planes, kernel_size, strides=strides, padding=padding, use_bias=use_bias),
        BatchNormalization(),
        ReLU()
    ])


def bottleneck(x, planes, stride=1, downsample=None, head_conv=1, use_bias=False):
    residual = x
    if head_conv == 1:
        x = Conv_BN_ReLU(planes, kernel_size=1, use_bias=use_bias)(x)
    elif head_conv == 3:
        x = Conv_BN_ReLU(planes, kernel_size=(3, 1, 1), use_bias=use_bias)(x)
    else:
        raise ValueError('Unsupported head_conv!!!')
    x = Conv_BN_ReLU(planes, kernel_size=(1, 3, 3), strides=(1, stride, stride), use_bias=use_bias)(x)
    x = Conv3D(planes*4, kernel_size=1, use_bias=use_bias)(x)
    x = BatchNormalization()(x)
    if downsample is not None:
        residual = downsample(residual)
    x = Add()([x, residual])
    x = ReLU()(x)
    return x

def datalayer(x, stride):
    return x[:, ::stride, :, :, :]

def SlowFast_body(inputs, layers, block, num_classes, dropout=0.5):
    inputs_fast = Lambda(datalayer, name='data_fast', arguments={'stride':2})(inputs)
    inputs_slow = Lambda(datalayer, name='data_slow', arguments={'stride':16})(inputs)
    fast, lateral = Fast_body(inputs_fast, layers, block)
    slow = Slow_body(inputs_slow, lateral, layers, block)
    x = Concatenate()([slow, fast])
    x = Dropout(dropout)(x)
    out = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, out)



def Fast_body(x, layers, block):
    fast_inplanes = 8
    lateral = []
    x = Conv_BN_ReLU(8, kernel_size=(5, 7, 7), strides=(1, 2, 2))(x)
    x = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same')(x)
    lateral_p1 = Conv3D(8*2, kernel_size=(5, 1, 1), strides=(8, 1, 1), padding='same', use_bias=False)(x)
    lateral.append(lateral_p1)
    x, fast_inplanes = make_layer_fast(x, block, 8, layers[0], head_conv=3, fast_inplanes=fast_inplanes)
    lateral_res2 = Conv3D(32*2, kernel_size=(5, 1, 1), strides=(8, 1, 1), padding='same', use_bias=False)(x)
    lateral.append(lateral_res2)
    x, fast_inplanes = make_layer_fast(x, block, 16, layers[1], stride=2, head_conv=3, fast_inplanes=fast_inplanes)
    lateral_res3 = Conv3D(64*2, kernel_size=(5, 1, 1), strides=(8, 1, 1), padding='same', use_bias=False)(x)
    lateral.append(lateral_res3)
    x, fast_inplanes = make_layer_fast(x, block, 32, layers[2], stride=2, head_conv=3, fast_inplanes=fast_inplanes)
    lateral_res4 = Conv3D(128*2, kernel_size=(5, 1, 1), strides=(8, 1, 1), padding='same', use_bias=False)(x)
    lateral.append(lateral_res4)
    x, fast_inplanes = make_layer_fast(x, block, 64, layers[3], stride=2, head_conv=3, fast_inplanes=fast_inplanes)
    x = GlobalAveragePooling3D()(x)
    return x, lateral

def Slow_body(x, lateral, layers, block):
    slow_inplanes = 64 + 64//8*2
    x = Conv_BN_ReLU(64, kernel_size=(1, 7, 7), strides=(1, 2, 2))(x)
    x = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same')(x)
    x = Concatenate()([x, lateral[0]])
    x, slow_inplanes = make_layer_slow(x, block, 64, layers[0], head_conv=1, slow_inplanes=slow_inplanes)
    x = Concatenate()([x, lateral[1]])
    x, slow_inplanes = make_layer_slow(x, block, 128, layers[1], stride=2, head_conv=1, slow_inplanes=slow_inplanes)
    x = Concatenate()([x, lateral[2]])
    x, slow_inplanes = make_layer_slow(x, block, 256, layers[2], stride=2, head_conv=1, slow_inplanes=slow_inplanes)
    x = Concatenate()([x, lateral[3]])
    x, slow_inplanes = make_layer_slow(x, block, 512, layers[3], stride=2, head_conv=1, slow_inplanes=slow_inplanes)
    x = GlobalAveragePooling3D()(x)
    return x


def make_layer_fast(x, block, planes, blocks, stride=1, head_conv=1, fast_inplanes=8, block_expansion=4):
    downsample = None
    if stride != 1 or fast_inplanes != planes * block_expansion:
        downsample = Sequential([
            Conv3D(planes*block_expansion, kernel_size=1, strides=(1, stride, stride), use_bias=False),
            BatchNormalization()
        ])
    fast_inplanes = planes * block_expansion
    x = block(x, planes, stride, downsample=downsample, head_conv=head_conv)
    for _ in range(1, blocks):
        x = block(x, planes, head_conv=head_conv)
    return x, fast_inplanes

def make_layer_slow(x, block, planes, blocks, stride=1, head_conv=1, slow_inplanes=80, block_expansion=4):
    downsample = None
    if stride != 1 or slow_inplanes != planes * block_expansion:
        downsample = Sequential([
            Conv3D(planes*block_expansion, kernel_size=1, strides = (1, stride, stride), use_bias=False),
            BatchNormalization()
        ])
    x = block(x, planes, stride, downsample, head_conv=head_conv)
    for _ in range(1, blocks):
        x = block(x, planes, head_conv=head_conv)
    slow_inplanes = planes * block_expansion + planes * block_expansion//8*2
    return x, slow_inplanes




def resnet50(inputs, **kwargs):
    model = SlowFast_body(inputs, [3, 4, 6, 3], bottleneck, **kwargs)
    return model

def resnet101(inputs, **kwargs):
    model = SlowFast_body(inputs, [3, 4, 23, 3], bottleneck, **kwargs)
    return model

def resnet152(inputs, **kwargs):
    model = SlowFast_body(inputs, [3, 8, 36, 3], bottleneck, **kwargs)
    return model

def resnet200(inputs, **kwargs):
    model = Slow_body(inputs, [3, 24, 36, 3], bottleneck, **kwargs)
    return model



network = {
    'resnet50':resnet50,
    'resnet101':resnet101,
    'resnet152':resnet152,
    'resnet200':resnet200
}


x = Input(shape=(None, None, None, 3))
model = resnet50(x, num_classes=15)

In [ ]:
#model = SlowFast_Network(clip_shape=[ModelConfig.SEQUENCE_SIZE,ModelConfig.H, ModelConfig.W, ModelConfig.C],
#                         num_class=ModelConfig.NUM_CLASSES,alpha=8,beta=1/8,tau=16,method='T_conv')
print(model.summary())

In [ ]:
from tensorflow.keras.optimizers import Adam

X_input = Input(shape=((ModelConfig.SEQUENCE_SIZE, ModelConfig.H, ModelConfig.W, ModelConfig.C)))
clf  = resnet50(X_input, 
                            num_classes=ModelConfig.NUM_CLASSES)
#opt = Adam(learning_rate=1e-3, beta_1 = 0.9, decay = 1e-4)
opt = Adam()
clf.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])   

In [ ]:
!pip install livelossplot

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from livelossplot import PlotLossesKeras


model_weights_file_path = os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.CLASSIFIER_MODEL_PATH)
checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max", save_weights_only=True)
early_stopping = EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, patience=20)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, verbose=0, mode='max', min_delta=0.0001, cooldown=0, min_lr=0)
plotlosses = PlotLossesKeras()

call_backs = [checkpoint, early_stopping, lr_reduce, plotlosses]



In [ ]:
# #with tf.device(device_name):
# history = clf.fit(train_gen, 
#                     validation_data=validation_gen,
#                     epochs=ModelConfig.EPOCHS, 
#                     callbacks=call_backs, 
#                     class_weight=class_weights,
#                     verbose=1)

In [ ]:
# clf_model.evaluate(train_gen)

In [ ]:
# X, y = train_gen.get_mini_batch(288)
# print(X[0][-1])
# read_npz_file(X[0][-1]).shape